In [ ]:
from neuralogic.nn import get_evaluator
from neuralogic.core import R, Template, V, Settings, Transformation, Combination, Aggregation
from neuralogic.core.constructs.function.tree import FunctionContainer
from neuralogic.core.settings import Settings
from neuralogic.dataset import Dataset
from neuralogic.optim import SGD



In [ ]:
F = FunctionContainer()     # new but necessary feature, necessary to use both round and square brackets for same purpose
train_dataset = Dataset()
template = Template()

train_dataset.add_example([R.b[3], R.c[5]])

Ukázka funkční nové syntaxe, kterou jsi po mně chtěl v tom nedávném emailu. Nyní fungují obě varianty stejně.

Poměrně obtížné bylo zajistit, aby hranaté závorky fungovaly stejně jako kulaté a zápis aby byl přehlednější. To se mi nakonec povedlo za cenu, že třída FunctionContainer z modulu neuralogic.core.constructs.function.tree je trochu složitější a hůře čitelná.

In [ ]:
# 1st option
template += (R.message2(V.X)[5,10] <= (x:= R.message1(V.Y)[10,20], y:= R.edge(V.Y, V.X))) >> F.avg(F.relu(x + y))

# 2nd option - round brackets
template += (R.message2(V.X)[5,10] <= F.avg(F.relu(R.message1(V.Y)[10,20] + R.edge(V.Y, V.X))))

# 2nd option - both round and square brackets
template += (R.message2(V.X)[5,10] <= F.avg[F.relu(R.message1(V.Y)[10,20] + R.edge(V.Y, V.X))])